# **1. Pyomo and Solvers Installation:**

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils   #glpk
!apt-get install -y -qq coinor-cbc   #cbc

from pyomo.environ import *

In [2]:
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"   #IPOPT
!unzip -o -q ipopt-linux64

In [ ]:
!pip install cplex -q   #cplex

# **2. Problem Definition:**

- **`ConcreteModel()`**: Used when the entire model is defined at once.
- **`AbstractModel()`**: Used when some parts of the model are defined later.

In [16]:
model = ConcreteModel()
model1 = ConcreteModel()
model2 = ConcreteModel()

# **3. Defining Variables:**

The **`Var()`** function is used to declare variables in a Pyomo model.

- `bounds`: The bounds parameter is used to specify the lower and upper limits for the variable. It is given as a tuple (lower_bound, upper_bound).
- `within`: This parameter is used to specify the domain of the variable. It defines the set of values the variable is allowed to take.
    - Common domains for variables in Pyomo:

      - `Reals`: Any real number (positive, negative, or zero).
      - `NonNegativeReals`: Real numbers that are greater than or equal to 0.
      - `NonPositiveReals`: Real numbers that are less than or equal to 0.
      - `Integers`: Whole numbers (positive, negative, or zero).
      - `NonNegativeIntegers`: Whole numbers greater than or equal to 0.
      - `Binary`: Variables that can only take the values 0 or 1 (often used in mixed-integer programming).

In [4]:
model.x = Var(bounds=(0, None), within=NonNegativeReals)
model.y = Var(bounds=(0, None), within=NonNegativeReals)

**Defining Indexed Variables:**

In Pyomo, you use Sets to define the indices and Var to define the decision variables.

In [5]:
# Define the index set
model1.indices = Set(initialize=['A', 'B', 'C'])

# Define the decision variables d[i] for each i in indices
# The decision variables have a lower bound of 0 and are continuous
model1.d = Var(model1.indices, domain=NonNegativeReals)

# Display the variables
model1.pprint()

1 Set Declarations
    indices : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}

1 Var Declarations
    d : Size=3, Index=indices
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  None :  None : False :  True : NonNegativeReals
          B :     0 :  None :  None : False :  True : NonNegativeReals
          C :     0 :  None :  None : False :  True : NonNegativeReals

2 Declarations: indices d


In [18]:
# Define the sets for crops and areas
model2.crops = Set(initialize=['sugar beets', 'cotton', 'sorghum'])
model2.areas = Set(initialize=[1, 2, 3])

# Define the decision variables x[crop, area]
model2.x = Var(model2.crops, model2.areas, domain=NonNegativeReals)

# Display the variables
model2.x.pprint()  # This will show the variable names and their properties

x : Size=9, Index=crops*areas
    Key                : Lower : Value : Upper : Fixed : Stale : Domain
         ('cotton', 1) :     0 :  None :  None : False :  True : NonNegativeReals
         ('cotton', 2) :     0 :  None :  None : False :  True : NonNegativeReals
         ('cotton', 3) :     0 :  None :  None : False :  True : NonNegativeReals
        ('sorghum', 1) :     0 :  None :  None : False :  True : NonNegativeReals
        ('sorghum', 2) :     0 :  None :  None : False :  True : NonNegativeReals
        ('sorghum', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('sugar beets', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('sugar beets', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('sugar beets', 3) :     0 :  None :  None : False :  True : NonNegativeReals


# **4. Objective Function:**

To define an objective function, you use the **`Objective()`** function within your model. The general structure for defining an objective function in Pyomo is:

`model.objective_name = Objective(expr=<expression>, sense=minimize/maximize)
`

In [6]:
model.obj = Objective(expr=2*model.x + 3*model.y, sense=minimize)

# **5. Constraints:**

The general syntax for defining a constraint in Pyomo is:

`model.constraint_name = Constraint(expr=<expression>)`

- expr: This represents the mathematical expression of the constraint, involving variables and constants.

- Comparison Operators:

  Inequalities: Use <=, >= for "less than or equal to" and "greater than or equal to."

  Equalities: Use == for equality constraints.

In [7]:
model.constraint1 = Constraint(expr=2*model.x + model.y >= 1)
model.constraint2 = Constraint(expr=model.x + 2*model.y >= 2)

In [8]:
# Define constraints for each index
def constraint_rule(model, i):
    return model1.d[i] >= 10

# Add constraints using the rule
model1.constraints = Constraint(model1.indices, rule=constraint_rule)

In [9]:
model1.constraints.pprint()

constraints : Size=3, Index=indices, Active=True
    Key : Lower : Body : Upper : Active
      A :  10.0 : d[A] :  +Inf :   True
      B :  10.0 : d[B] :  +Inf :   True
      C :  10.0 : d[C] :  +Inf :   True


# **6. Solving the Problem:**

After defining the model, you solve it using a solver.

You can use various solvers depending on the problem:

- Linear problems: glpk, cbc, gurobi, etc.
- Nonlinear problems: ipopt, knitro, etc.

In [10]:
SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 3.0, 'Upper bound': 3.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of nonzeros': 4, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.011139392852783203}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
SolverFactory('cplex_direct').solve(model)

# **7. Extracting Results:**

In [11]:
model.display()

Model unknown

  Variables:
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   3.0

  Constraints:
    constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :   1.0 :  1.0 :  None
    constraint2 : Size=1
        Key  : Lower : Body : Upper
        None :   2.0 :  2.0 :  None


In [15]:
# Get all variable names
variable_names = model.component_objects(Var, active=True)

# Print all variable names and their values
for var in variable_names:
    print(f"Variable name: {var.name}, Value: {var.value}")

Variable name: x, Value: 2.00000000175489
Variable name: y, Value: 3.00000000142049


In [12]:
# Display the results of the variables
print("Variable values:")
print(f"x = {model.x.value}")
print(f"y = {model.y.value}")

Variable values:
x = 0.0
y = 1.0


In [13]:
print("Variable values:")
model1.d.pprint()

Variable values:
d : Size=3, Index=indices
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :  None :  None : False :  True : NonNegativeReals
      B :     0 :  None :  None : False :  True : NonNegativeReals
      C :     0 :  None :  None : False :  True : NonNegativeReals


# **8. Nonlinear Example:**

In [14]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(bounds=(0, None))
model.y = Var(bounds=(0, None))

# Define objective function (nonlinear)
model.obj = Objective(expr=(model.x - 2)**2 + (model.y - 3)**2, sense=minimize)

# Define constraints (one of them is nonlinear)
model.constr1 = Constraint(expr=model.x**2 + model.y >= 2)
model.constr2 = Constraint(expr=model.x + model.y**2 >= 1)

# Solve the model
SolverFactory('ipopt', executable='ipopt').solve(model)

# Display results
model.display()

Model unknown

  Variables:
    x : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 2.00000000175489 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 3.00000000142049 :  None : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5.097430643062936e-18

  Constraints:
    constr1 : Size=1
        Key  : Lower : Body             : Upper
        None :   2.0 : 7.00000000844005 :  None
    constr2 : Size=1
        Key  : Lower : Body              : Upper
        None :   1.0 : 11.00000001027783 :  None
